In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
def transform_residuals(params, std_norm_e):
    std_r,std_x,std_f, corr_xf, corr_rx, corr_rf = params[['std_r', 'std_x', 'std_f', 'corr_xf', 'corr_rx', 'corr_rf']]
    cov = [[std_r**2, corr_rx*std_r*std_x, corr_rf*std_r*std_f],
        [corr_rx*std_r*std_x, std_x**2, corr_xf*std_x*std_f],
        [corr_rf*std_r*std_f, corr_xf*std_x*std_f, std_f**2]]
    cholesky_cov = np.linalg.cholesky(cov)
    e=cholesky_cov.dot(std_norm_e.T).T.reshape(T, sample_size, 3)
    return e[:params['T'], :params['n'],:]


In [12]:
def sim_VAR(params, e):
    
    growth_factor = params['growth_factor']
    r_mean = params['r_mean']
    mu = np.log(1+r_mean)
    T, sample_size = params['T'], params['n']
    std_r = params['std_r']
    a_1, a_2, a_3, b_0, b_1, b_2 = params[['a_1', 'a_2', 'a_3', 'b_0', 'b_1', 'b_2']]
    
    r = np.zeros((T, sample_size))
    X = np.zeros((T, sample_size))
    F = np.zeros((T, sample_size))

    #first simulate r because it is independent
    r = mu -.5*std_r**2 + e[:,:,0]
    #then simulate X and F
    #note that the first couple of values are taken from end of arrays
    for t in range(T):
        X[t] = a_1*r[t-1] + a_2*X[t-1] + a_3*X[t-2] + e[t,:,1]*growth_factor**t
        F[t] = b_0*r[t-1] + b_1*F[t-1] + b_2*F[t-2] + e[t,:,2]*growth_factor**t
    return r, X, F

In [13]:
#simulation function
def sim(A_0, X, F, param, new_smoothing):
    df = pd.DataFrame(index = ['5%', '25%', '50%', '75%', '95%', 'Mean','Depletion rate']) #table to save results to


    lambda_ = param['lambda']
    r_mean = param['r_mean']
    T, sample_size = param['T'], param['n']
    A = np.zeros((T, sample_size))
    D = np.zeros((T, sample_size))
    S = np.zeros((T, sample_size))
    A[0] = param['A_0']
    S[0] = r_mean*A[0]
    D[0] = S[0] + X[0]*param['X'] + F[0]*param['F']
    for t in range(1,T):
        A[t] = np.maximum(A[t-1]*np.exp(r[t-1])-D[t-1], 0)

        if new_smoothing:
            S[t] = np.where(S[t-1]<=r_mean*A[t], r_mean*A[t]+F[t]*param['F'], lambda_*S[t-1]+(1-lambda_)*r_mean*A[t]+F[t]*param['F'])
            #S[t] = np.maximum(r_mean*A[t]+F[t]*param['F'], lambda_*S[t-1]+(1-lambda_)*r_mean*A[t]+F[t]*param['F'])
        else:    
            S[t] = np.where(S[t-1]<=r_mean*A[t-1], r_mean*A[t]+F[t]*param['F'], lambda_*S[t-1]+(1-lambda_)*r_mean*A[t]+F[t]*param['F']) 
        
        D[t] = S[t] + X[t]*param['X']

    df[r'$\lambda$={}, X={}, F={}'.format(param['lambda'], param['X'], param['F'])] = [np.percentile(A[-1], 5),np.percentile(A[-1], 25), np.percentile(A[-1], 50), np.percentile(A[-1], 75), np.percentile(A[-1], 95), A[-1].mean(),(A[-1]==0).mean()]
    return df

In [19]:
sim_param = pd.read_excel('sim_param.xlsx', index_col = 0, dtype = {'n':'int32', 'T':'int32', 'X': 'bool', 'A_0': 'float32'})
#generate random errors
sample_size = sim_param['n'].max()
T = sim_param['T'].max()
std_norm_e = np.random.standard_normal(size=(T*sample_size, 3))

In [20]:
sim_param[['5%', '25%', '50%', '75%', '95%', 'Mean','Depletion rate']] = 0.
for i in range(len(sim_param)):
    e = transform_residuals(sim_param.iloc[i], std_norm_e)
    r, X, F = sim_VAR(sim_param.iloc[i], e)
    A_0, new_smoothing = sim_param.iloc[i][['A_0', 'new_smoothing']]
    df = sim(A_0, X, F, sim_param.iloc[i], new_smoothing)
    sim_param.iloc[i,-7:] = df.values.reshape(-1)
    print('Simulation', str(i+1), 'of', str(len(sim_param)), 'done!')

Simulation 1 of 40 done!
Simulation 2 of 40 done!
Simulation 3 of 40 done!
Simulation 4 of 40 done!
Simulation 5 of 40 done!
Simulation 6 of 40 done!
Simulation 7 of 40 done!
Simulation 8 of 40 done!
Simulation 9 of 40 done!
Simulation 10 of 40 done!
Simulation 11 of 40 done!
Simulation 12 of 40 done!
Simulation 13 of 40 done!
Simulation 14 of 40 done!
Simulation 15 of 40 done!
Simulation 16 of 40 done!
Simulation 17 of 40 done!
Simulation 18 of 40 done!
Simulation 19 of 40 done!
Simulation 20 of 40 done!
Simulation 21 of 40 done!
Simulation 22 of 40 done!
Simulation 23 of 40 done!
Simulation 24 of 40 done!
Simulation 25 of 40 done!
Simulation 26 of 40 done!
Simulation 27 of 40 done!
Simulation 28 of 40 done!
Simulation 29 of 40 done!
Simulation 30 of 40 done!
Simulation 31 of 40 done!
Simulation 32 of 40 done!
Simulation 33 of 40 done!
Simulation 34 of 40 done!
Simulation 35 of 40 done!
Simulation 36 of 40 done!
Simulation 37 of 40 done!
Simulation 38 of 40 done!
Simulation 39 of 40 d

In [21]:
sim_param.T.to_excel('sim_results.xlsx')